In [1]:
%load_ext autoreload
%autoreload 2

In [51]:
import sys_pdf
import pdf_functions as pdf
import os
import sys_pdf

'''
Process a PDF using search dictionary
'''

def open_and_read(working_dir, file_name):# open a file and read
    try: 
        path_name = working_dir + "/" + file_name
        scanned_text = pdf.read_pdf(path_name)
    except:
        sys_pdf.sys_log_entry('read error', "Could not open file: " + path_name)
        sys_pdf.move_file(working_dir, "not_processed", file_name)
    return scanned_text, path_name


def format_text(scanned_text):
    # add the contents for each page into a list
    contents = pdf.text_to_list(scanned_text)

    # iterate over each page
    by_line = []
    for page in range(0, len(scanned_text.pages)):  # create a flat list of strings
        by_line += contents[page].split('\n')       # remove the line breaks
    return by_line


def pull_name_date(overview_table):
    company_name = overview_table['invoice_company'][0].replace('.', '_')
    # format dates to standard YYYY-MM-DD
    formatted_date_str = pdf.format_date(overview_table['invoice_date'][0], '%B %d, %Y') # November 11, 2015
    return company_name, formatted_date_str

def save_name(company_name, formatted_date_str):
    name_base = company_name + "_" + formatted_date_str
    return name_base

def process_pdfs(working_dir, csv_dir):
    file_counter = 0
     
    # for filename in working_dir
    for file_name in os.listdir(working_dir):
    
        # pull text from pdf file
        scanned_text, path_name = open_and_read(working_dir, file_name)
        
        # process the file
        try: 
            # make a list of text
            by_line = format_text(scanned_text)
            
            # open dictionary of search terms
            table_1_search, table_1_type = pdf.table_1_search_dict()
            table_2_search, table_2_type = pdf.table_2_search_dict()
        
            # create tables with pulled data
            table_1 = pdf.one_table(table_1_search, by_line)
            table_2 = pdf.many_table(table_2_search, by_line)  
        
            # 
            table_1_df = pdf.one_table_df(table_1)
            table_2_df = pdf.many_table_df(table_2)  # verifies that the length of each value list is the same in a many table
        
            # add a column of invoice numbers for each table_2 entry
            #table_2_df['invoice_number'] = table_1['invoice_number'][0]
            table_2_df.insert(0, 'invoice_number', table_1['invoice_number'][0])

            # grab company name and invoice date from overview table
            company_name, invoice_date = pull_name_date(table_1)
            name_base = save_name(company_name, invoice_date)
            
            # save the csv and log
            table_dict = {"overview":table_1_df, "products":table_2_df}
            for key, value in table_dict.items():
                routine,log_value=pdf.save_as_csv(value, csv_dir, name_base, key)
                sys_pdf.sys_log_entry(routine, log_value)
            
            # save the pdf in the 'processed' directory
            sys_pdf.move_file(working_dir, "processed", file_name)
            
            file_counter += 1
            
        except:
            sys_pdf.sys_log_entry('production error', "Could not properly analyze file: " + path_name)
            #  move pdf to error directory
            sys_pdf.move_file(working_dir, "not_processed", file_name)
    
    print(f"A total of {file_counter} PDF files were processed.")

In [52]:
# Move PDF files to working directory
source_dir = "pdf_inbox"
working_dir = "working_dir"
csv_dir = "example_output_files"

files_present = sys_pdf.to_working_dir(source_dir, working_dir)

if files_present:   # only move forward with formatting if there are files to process
    process_pdfs(working_dir, csv_dir)

move Files in inbox: 1. Moved to working dir: 1. 
The file has 2 pages.
CSV OK. Saved: overview Amazon_com_2015-11-11
CSV OK. Saved: products Amazon_com_2015-11-11
A total of 1 PDF files were processed.


In [94]:
'''
Build a search dictionary.
'''
import pandas as pd

# Move PDF files to working directory
source_dir = "NEW_PDF_style"
working_dir = "working_dir"
search_file_name = "amazon111924.xlsx"

# checks directory if is a new PDF file to be formatted, if s move to working dir
files_present = sys_pdf.to_working_dir(source_dir, working_dir)

if files_present:
    # open excel file as a pandas dataframe
    guide = pd.read_excel("search_guides/"+search_file_name)
    
    # table headers from the guide
    overview_table_headers = [ 'Invoice date', 'Invoice number', 'Company name', 'Invoice total' ]
    products_table_headers = [ 'Product name', 'Product company', 'Product amount' ]

    # the name of the pdf file from search guide
    pdf_file = guide['File name of PDF'][0]
    
    # open and pull text from pdf file
    scanned_text, path_name = open_and_read(working_dir, pdf_file)

    # prep the text strings for scanning
    by_line = format_text(scanned_text)

    # iterate over each table header
    #for header in overview_table_headers:
    header = guide['Invoice number'][0]
    
    # iterate over each line in the list of text strings
    for line in by_line:
        # is there a match between the search string and the line?
        # if so return the match
        #matching_string = pdf.find_string(line, header, position, del_chars)

    if matching_string != None:
        # clean up the string
        clean_string = clean_text(matching_string)

move Files in inbox: 1. Moved to working dir: 1. 
The file has 2 pages.


In [100]:
guide['Invoice number'][0]

'104-1523806-7547463'

In [136]:
pos=0
header = guide['Invoice number'][0]
len_target_text = len(header)
for line in by_line:
    char_location = line.find(header)
    if char_location >= 0: 
        the_line = line
        pos=char_location
        break


In [137]:
the_line

'Details for Orde r #104-1523806-7547463 '

In [138]:
pos

20

In [132]:
# The search term 
front = the_line[0:pos]

back = the_line[pos+len_search_str:]

In [139]:
the_line[0:pos]

'Details for Orde r #'

In [53]:
def extract_text_from_pdf(file_path):
    """Extracts text from a PDF file."""
    text = ""
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

def extract_invoice_data(client, text):
    """Extracts specific invoice details using OpenAI ChatCompletion API."""
    # Prompt for the model to identify relevant data
    prompt = f"""
    Extract the following details from the text:
    - Invoice Date
    - Invoice Number
    - Invoice Company
    - Invoice Total
    Text: {text}
    Provide the data in a JSON format with keys: invoice_date, invoice_number, invoice_company, invoice_total.
    """
    #client.completions.create()
    response = client.chat.completions.create(
        max_tokens = 2000,
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that extracts structured data from text."},
            {"role": "user", "content": prompt}
        ]
    )
    extracted_data = response.choices[0].message.content.strip("```json").replace("\n","")
    return response, eval(extracted_data)  # Convert string to dictionary

def create_invoice_table(data):
    """Creates a table with the extracted data."""
    df = pd.DataFrame([data])
    #df.columns = ["Invoice Date", "Invoice Number", "Invoice Company", "Invoice Total"]
    return df


                

In [54]:
'''
Use openai api to extract data from a PDF file and format into a table.
'''

import PyPDF2
import pandas as pd
import dotenv
import os
import openai
import pdf_functions as pdf

# Load environment variables from the .env file
dotenv.load_dotenv()

# instantiate 
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# File path to the PDF
file_path = "working_dir/amazon_111215.pdf"

# Step 1: Extract text from the PDF
pdf_text = extract_text_from_pdf(file_path)

# Step 2: Extract specific invoice details using OpenAI ChatCompletion
response, invoice_data = extract_invoice_data(client, pdf_text)

# Step 3: Create a pandas dataframe with the extracted data
invoice_table = create_invoice_table(invoice_data)

# Step 4: Save the dataframe to a CSV file
#invoice_table.to_csv("invoice_table.csv", index=False)  # original

# grab company name and invoice date from overview table
company_name, invoice_date = pull_name_date(invoice_table)
name_base = save_name(company_name, invoice_date)
            
# save the csv and log
routine,log_value=pdf.save_as_csv(value, csv_dir, name_base, key)
#sys_pdf.sys_log_entry(routine, log_value)

FileNotFoundError: [Errno 2] No such file or directory: 'working_dir/amazon_111215.pdf'